#0. 登録制度東京都申請フォームの入力内容を整理
申請されたデータをExcelで出力したのち、csvに変換し、処理を容易に

In [ ]:
# 作業用ファイルの指定
# グーグルフォームへの申請内容(Excel)
form_pass = '登録制度東京都申請フォーム（案）（回答）.xlsx'

# グーグルフォームへの申請内容(csv)
# form_pass = '登録制度東京都申請フォーム（案）（回答） - フォームの回答 1 (1).csv'

# カラム名修正用のcsvファイル
column_name_pass = 'column_name.csv'

In [ ]:
# パッケージの読み込み(下準備）
import pandas as pd
from datetime import datetime, timezone, timedelta
import json

In [ ]:
# 処理開始時間を取得
start_time = datetime.now(timezone(timedelta(hours=9)))
# 処理開始時間を日本時間に設定
start_time = start_time.astimezone(timezone(timedelta(hours=9)))
# 処理開始時間をYYYYMMDDHHMMSS形式に変換
start_time_str = start_time.strftime('%Y%m%d%H%M%S')
# 処理開始時間を表示
print('カラム名修正処理_開始時間:', start_time_str)

In [ ]:
# グーグルフォームへの申請内容(Excel)のシート名
form_sheet_name = 'フォームの回答 1'

# グーグルフォームへの申請内容(Excel)を読み込む
form_df = pd.read_excel(form_pass, sheet_name=form_sheet_name)

# 修正後のカラム名がリストされているカラム名修正用のcsvファイルを読み込む
column_name_df = pd.read_csv(column_name_pass, encoding='utf-8')

# 読み込み完了のメッセージを表示
print('必要なデータを読み込みました。')

In [ ]:
# カラム名修正用のデータとグーグルフォームへの申請内容のカラム数を照合
if len(column_name_df) != len(form_df.columns):
    print('カラム名修正用のデータとグーグルフォームへの申請内容のカラム数が一致しません。')
    print('カラム名修正用のデータ:', len(column_name_df), 'カラム')
    print('グーグルフォームへの申請内容:', len(form_df.columns), 'カラム')
    exit()
# カラム名修正用のデータをリストに変換
column_name_list = column_name_df['修正後のカラム名'].tolist()
# グーグルフォームへの申請内容のカラム名を修正後のカラム名に変更
form_df.columns = column_name_list
# 修正後のカラム名を表示
print('修正後のカラム名:', form_df.columns.tolist())
# 修正後のデータをCSVファイルに保存
form_df.to_csv('申請内容_' + start_time_str + '.csv', index=False, encoding='utf-8-sig')

# 修正完了のメッセージを表示
print('申請内容のカラム名を修正し、CSVファイルに保存しました。')

#1. 登録制度東京都申請フォームの自動受付
申請されたデータをもとに、まず申請状況をリスト化

In [ ]:
# パッケージの読み込み(下準備）
import os
import pandas as pd

In [ ]:
# クラブ名のリスト(csv):f'クラブリスト_{YYYYMMDDHHMMSS}.csvの形式で保存されているので、最新のクラブリストを読み込む
try:
    # クラブ名のクラブリスト_{YYYYMMDDHHMMSS}.csvのファイルを探し、最新のものを読み込む
    club_list_files = [f for f in os.listdir('.') if f.startswith('クラブリスト_') and f.endswith('.csv')]
    if not club_list_files:
        raise FileNotFoundError("クラブリストのCSVファイルが見つかりません。")
    # 最新のクラブリストファイルを取得
    club_list_file_for_acceptance = max(club_list_files, key=os.path.getctime)
    # 最新のクラブリストを読み込む
    club_list_df_for_acceptance = pd.read_csv(club_list_file_for_acceptance, encoding='utf-8-sig')
    # クラブ名のリストを確認
    print(club_list_df_for_acceptance.head())
    print("クラブリストの読み込みが完了しました")
except Exception as e:
    print(f"クラブリストの読み込み中にエラーが発生しました: {e}")

In [ ]:
# クラブからの申請内容が申請内容_{start_time_str}.csvの形式で保存されているので、最新の申請内容を読み込む
try:
    # クラブ名の申請内容_{start_time_str}.csvのファイルを探し、最新のものを読み込む
    form_files = [f for f in os.listdir('.') if f.startswith('申請内容_') and f.endswith('.csv')]
    if not form_files:
        raise FileNotFoundError("申請内容のCSVファイルが見つかりません。")
    # 最新の申請内容ファイルを取得
    form_file_for_acceptance = max(form_files, key=os.path.getctime)
    # 最新の申請内容を読み込む
    form_df_for_acceptance = pd.read_csv(form_file_for_acceptance, encoding='utf-8-sig')
    # 申請内容のデータを確認
    print(form_df_for_acceptance.head())
    print(form_df_for_acceptance.columns)
    print(form_df_for_acceptance.dtypes)
    print("申請内容の読み込みが完了しました")
except Exception as e:
    print(f"申請内容の読み込み中にエラーが発生しました: {e}")

# タイムスタンプをdatetime型に変換
form_df_for_acceptance['タイムスタンプ'] = pd.to_datetime(form_df_for_acceptance['タイムスタンプ'], errors='coerce')
form_df_for_acceptance['タイムスタンプyyyymmddHHMMSS'] = form_df_for_acceptance['タイムスタンプ'].dt.strftime('%Y%m%d%H%M%S')

In [ ]:
# club_list_dfに申請内容を基に申請した追記
# 最新の申請のみを抽出（クラブ名ごとに最新のタイムスタンプ）
latest_applied_clubs_df = form_df_for_acceptance.sort_values('タイムスタンプ').drop_duplicates('クラブ名を選択してください', keep='last')

# クラブ名でマージ
merged_club_list_df_for_acceptance = pd.merge(
    club_list_df_for_acceptance,
    latest_applied_clubs_df,
    left_on='選択肢（地区名：クラブ名：クラブ名（カタカナ））',
    right_on='クラブ名を選択してください',
    how='left'
    )

# 申請フラグとタイムスタンプ列を追加
merged_club_list_df_for_acceptance['R8年度登録申請クラブ'] = merged_club_list_df_for_acceptance['タイムスタンプ'].notna().astype(int)
merged_club_list_df_for_acceptance['R8年度登録申請_タイムスタンプ'] = merged_club_list_df_for_acceptance['タイムスタンプ']

# 結果確認
print(merged_club_list_df_for_acceptance[['選択肢（地区名：クラブ名：クラブ名（カタカナ））', 'R8年度登録申請クラブ', 'R8年度登録申請_タイムスタンプ']].head())

In [ ]:
# merged_club_list_dfを'クラブリスト_yyyymmddhhmmss.csv'に出力
# 日本の現在時刻を取得
jst_now = datetime.now(timezone(timedelta(hours=9)))

# 時刻をyyyymmddHHMMSS形式に変換
timestamp_for_acceptance = jst_now.strftime("%Y%m%d%H%M%S")

# 出力時のファイル名を指定
output_file_path_for_acceptance = f'申請受付リスト_{timestamp_for_acceptance}.csv'

# csvを出力
merged_club_list_df_for_acceptance.to_csv(output_file_path_for_acceptance, index=False)
print(f"出力ファイルパス: {output_file_path_for_acceptance}")

#2. クラブごとチェックリスト作成

申請されたデータをもとにクラブごとのチェックリストを作成

In [ ]:
# パッケージの読み込み(下準備）
import pandas as pd
import os
from datetime import datetime, timezone, timedelta

In [ ]:
# 日本の現在時刻を取得
jst_now = datetime.now(timezone(timedelta(hours=9)))

# 時刻をyyyymmddHHMMSS形式に変換
timestamp_for_make_checklist = jst_now.strftime("%Y%m%d%H%M%S")

In [ ]:
# クラブリスト_{timestamp}.csvを読み込む
# folderを参照して、最新のクラブリストを探す
foldr_path_for_make_checklist = 'R7_登録申請処理/クラブリスト'
latest_file = max([os.path.join(foldr_path_for_make_checklist, f) for f in os.listdir(foldr_path_for_make_checklist)], key=os.path.getctime)

# 最新のクラブリストを読み込む
club_list_df_for_make_checklist = pd.read_csv(latest_file)
print(club_list_df_for_make_checklist.head())
print(club_list_df_for_make_checklist.columns)
print(club_list_df_for_make_checklist.dtypes)
print("クラブリストの読み込みが完了しました")

# R8年度登録申請クラブが1のデータのみ保持
application_club_list_df_for_make_checklist = club_list_df_for_make_checklist[club_list_df_for_make_checklist['R8年度登録申請クラブ'] == 1]

In [ ]:
# タイムスタンプをdatetime型に変換
application_club_list_df_for_make_checklist['タイムスタンプ'] = pd.to_datetime(application_club_list_df_for_make_checklist['タイムスタンプ'], errors='coerce')

# さらに、yyyymmddHHMMSS形式に変換
application_club_list_df_for_make_checklist['タイムスタンプyyyymmddHHMMSS'] = application_club_list_df_for_make_checklist['タイムスタンプ'].dt.strftime('%Y%m%d%H%M%S')

In [ ]:
# 出力フォルダの作成（作成済みの場合は処理をパス）
if not os.path.exists('R7_登録申請処理/申請入力内容'):
    os.makedirs('R7_登録申請処理/申請入力内容',exist_ok=True)
    output_folder = 'R7_登録申請処理/申請入力内容'
    print('フォルダが作成されました')
else:
    output_folder = 'R7_登録申請処理/申請入力内容'
    print('フォルダは既に存在しています')

In [ ]:
# クラブごとのチェックリスト作成状況を保存するcsvを読み込み（ない場合は作成）
if os.path.exists('R7_登録申請処理/申請入力内容/クラブごとのチェックリスト作成状況.csv'):
  checklist_create_df = pd.read_csv('R7_登録申請処理/申請入力内容/クラブごとのチェックリスト作成状況.csv')
  print('クラブごとのチェックリスト作成状況.csvはすでに存在しています')
else:
  checklist_create_df = pd.DataFrame(columns=['クラブ名','申請日時', 'チェックリスト作成日時'])
  checklist_create_df.to_csv('R7_登録申請処理/申請入力内容/クラブごとのチェックリスト作成状況.csv', index=False)
  print('クラブごとのチェックリスト作成状況.csvが作成されました')

In [ ]:
# クラブごとのチェックリストを作成
# クラブごとの処理
for _, row in application_club_list_df_for_make_checklist.iterrows():
  try:
    club_data = {
        'クラブ名': [row['クラブ名']],
        '地区名': [row['地区名']],
        '担当者名': [row['申請担当者の氏名を入力してください']],
        '役職名': [row['申請担当者の役職名を入力してください']],
        'メールアドレス': [row['メールアドレス']],
        '電話番号': [row['申請担当者の日中連絡がつく電話番号を入力してください']],
        'FAX番号': [row['（必要があれば）申請担当者のFAX番号を入力してください']],
        '申請時間': [row['タイムスタンプ']],
        '自動チェック': [''],
        '自動チェック更新時間': [''],
        '書類チェック': [''],
        '書類チェック更新時間': [''],
        '書類間チェック': [''],
        '書類間チェック更新時間': [''],
        '担当者登録基準最終チェック': [''],
        '担当者登録基準最終チェック更新時間': [''],
    }
    club_df_for_make_checklist = pd.DataFrame(club_data)
    print(f"{row['クラブ名']}のclub_dfが作成されました")
    # checklist_create_dfにクラブ名があるかを確認
    if row['クラブ名'] not in checklist_create_df['クラブ名'].values:
      print('クラブ名がまだ存在しません')
      # checklist_create_dfにクラブ名の列を追加
      new_row = pd.DataFrame([{
          'クラブ名': row['クラブ名'],
          '申請日時': row['タイムスタンプ'],
          'チェックリスト作成日時': timestamp_for_make_checklist
          }])
      checklist_create_df = pd.concat([checklist_create_df, new_row], ignore_index=True)
      print(f"{row['クラブ名']}の列が追加されました")
    else:
      print('クラブ名がすでに存在します')

    # checklist_create_dfにある申請日時と申請時間を照合
    print('checklist_create_dfにある申請日時と申請時間を照合に移行します')
    if row['タイムスタンプ'] == checklist_create_df.loc[checklist_create_df['クラブ名'] == row['クラブ名'], '申請日時'].values[0]:
      # その後の処理
      # csvに出力
      file_name = f"{row['クラブ名']}_申請{row['タイムスタンプyyyymmddHHMMSS']}_作成{timestamp_for_make_checklist}.csv"
      print(f"file_name: {file_name}")
      print(f"output_folder: {output_folder}")
      file_path = os.path.join(output_folder, file_name)
      print(f"file_path: {file_path}")
      print(f"出力ファイルパス: {file_path}")
      club_df_for_make_checklist.to_csv(file_path, index=False)
      # checklist_create_dfのアップデート
      checklist_create_df.loc[checklist_create_df['クラブ名'] == row['クラブ名'], '申請日時'] = row['タイムスタンプ']
      checklist_create_df.to_csv('R7_登録申請処理/申請入力内容/クラブごとのチェックリスト作成状況.csv', index=False)
      print(f"{row['クラブ名']}の申請日時が更新されました")
    else:
      print('申請日時が同じチェックリストを作成済みです')
  except Exception as e:
    print(f"クラブ {row['クラブ名']} の処理中にエラーが発生しました: {e}")

#3. クラブごとに自動チェック
申請されたデータをもとに記入漏れ等をチェック

In [ ]:
import pandas as pd
import os
from datetime import datetime, timezone, timedelta

In [ ]:
# 必須項目に入力があるかをチェックする関数
def check_must_columns(application_row):
    """
    必須の項目が記入されているかをチェックします。
    """
    error_dict = {}
    # 必須ではない項目を指定（変更があればここを修正）
    not_must_column = [
        '所在する区市町村を通じ、東京都に総合型クラブ設立の「届出」を提出中であることを確認できる資料を提出してください。',
        '（必要があれば）事務局が所在する建物名を入力してください',
        '（必要があれば）申請担当者のFAX番号を入力してください',
        "（上のリストにおいて、「このリストに含まれていない種目」を「実施している」と回答した方に伺います）「リストに含まれていない種目」のうち、定期的に行っているスポーツ活動種目の数を入力してください（※定期的とは年間12回以上行っていることを指します）",
        "（上のリストにおいて、「このリストに含まれていない種目」を「実施している」と回答した方に伺います）「リストに含まれていない種目」のうち、定期的に行っているスポーツ活動種目名を入力してくださいなお、入力時は種目名を半角カンマ（,）で区切って入力してください例：サッカー,野球,バスケ※定期的とは年間12回以上行っていることを指します",
        "規約等の改廃を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。",
        '改廃前の規約等をアップロードしてください',
        '規約等をアップロードしてください',
        '提出する役員名簿（議決権保有者名簿）の種類を入力してください',
        '役員名簿（議決権保有者名簿）をアップロードしてください',
        '事業報告を決議した意思決定機関の名称を入力してください',
        '事業報告の決議日について入力してください',
        '決議された事業報告をアップロードしてください',
        '事業報告を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。',
        '決算を決議した意思決定機関の名称を入力してください',
        '決算の決議日について入力してください',
        '決議された決算をアップロードしてください',
        '決算を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。'
    ]

    # 必須のカラムを指定（全column-not_must_column)
    must_column = [col for col in application_row.columns if col not in not_must_column]

    # 必須のカラムが空白（NaN）ではないかをチェック
    if application_row[must_column].isna().any().any():
        error_at_must_column = []
        for col in must_column:
            if not application_row.empty and pd.isna(application_row[col].iloc[0]):
                 error_at_must_column.append(col)
        if error_at_must_column:
            error_dict['e_a_000'] = f'必須の項目が入力されていない:{error_at_must_column}'
    else:
        print('必須項目が入力されている')

    return error_dict

In [ ]:
# 届け出中のクラブが資料を提出しているかをチェックする関数
def check_submitting_now(form_row):
    """
    form_row['クラブ名を選択してください']が「この中に無い」の時に
    form_row['所在する区市町村を通じ、東京都に総合型クラブ設立の「届出」を提出中であることを確認できる資料を提出してください。']にデータがあるかをチェックします。
    """
    error_dict = {}
    # form_rowが空でないことを確認
    if not form_row.empty:
        # form_row['クラブ名を選択してください']が'この中に無い'の時に専用のフラグを立てる
        flag_submitting_now = 0 # 初期値を0に設定
        if (form_row['クラブ名を選択してください'] == 'この中に無い').any():
            flag_submitting_now = 1 # 条件を満たす行が1つでもあればフラグを1に設定
            print('このクラブは東京都に「届出」提出中です')
        else:
            flag_submitting_now = 0 # 条件を満たす行がなければフラグは0のまま
            print('このクラブは東京都に「届出」済です')

        if flag_submitting_now == 1 and pd.isna(form_row['所在する区市町村を通じ、東京都に総合型クラブ設立の「届出」を提出中であることを確認できる資料を提出してください。'].iloc[0]):
            error_dict['e_a_001'] = '非届出クラブだが、都に「届出」中であると確認できる資料が提出されていない'
        else:
            print('クラブ名の記載に問題がない')
    else:
        print("警告: check_submitting_now 関数に空のform_rowが渡されました。")

    return error_dict

In [ ]:
# 電話番号をチェックする関数
def check_phone_number(form_row):
    """
    form_row['申請担当者の日中連絡がつく電話番号を入力してください']が電話番号形式かをチェックします。
    （ハイフンを除いて10桁もしくは11桁であるかをチェック）
    """
    error_dict = {}
    if not form_row.empty:
        phone_number = form_row['申請担当者の日中連絡がつく電話番号を入力してください'].iloc[0]
        # NaNでないことを確認
        if not pd.isna(phone_number):
             # 電話番号からハイフンを取り除く
            phone_number_cleaned = str(phone_number).replace('-', '')
            if not (len(phone_number_cleaned) == 10 or len(phone_number_cleaned) == 11):
                error_dict['e_a_002'] = '電話番号が不正である（ハイフンを除いて10桁もしくは11桁ではない）'
            else:
                print('電話番号の記載に問題がない')
        else:
            error_dict['e_a_002'] = '電話番号が入力されていない'
    else:
        print("警告: check_phone_number 関数に空のform_rowが渡されました。")
    return error_dict

In [ ]:
# FAX番号をチェックする関数
def check_fax_number(form_row):
    """
    form_row['（必要があれば）申請担当者のFAX番号を入力してください']が電話番号形式かをチェックします。
    （ハイフンを除いて10桁であるかをチェック）
    """
    error_dict = {}
    if not form_row.empty:
        fax_number = form_row['（必要があれば）申請担当者のFAX番号を入力してください'].iloc[0]
        # NaNでないことを確認
        if not pd.isna(fax_number):
            # FAX番号からハイフンを取り除く
            fax_number_cleaned = str(fax_number).replace('-', '')
            if not len(fax_number_cleaned) == 10:
                error_dict['e_a_003'] = 'FAX番号が不正である（ハイフンを除いて10桁ではない）'
            else:
                print('FAX番号の記載に問題がない')
        else:
            print('FAX番号が空白である')
    else:
        print("警告: check_fax_number 関数に空のform_rowが渡されました。")
    return error_dict

In [ ]:
# 申請種別を確認する関数
def check_application_type(application_row, application_data_df, club_name):
    """
    application_row['申請種別を選択してください']が application_data_df のR7年度の登録状況と一致するかをチェックします。
    """
    error_dict = {}
    if not application_row.empty:
        # application_data_df に現在のクラブ名に対応する行があるかを確認
        application_data_match_row = application_data_df[application_data_df['クラブ名を選択してください'] == club_name]
        if not application_data_match_row.empty:
            if application_row['申請種別を選択してください'].iloc[0] == '新規（R7年度には登録していない）' and application_data_match_row['R7年度登録クラブ'].iloc[0] == 1:
                error_dict['e_a_004-a'] = 'R7年度の登録クラブだが、新規登録として申請されている'
            elif application_row['申請種別を選択してください'].iloc[0] == '更新（R7年度に登録済み）' and application_data_match_row['R7年度登録クラブ'].iloc[0] == 0:
                error_dict['e_a_004-b'] = 'R7年度の登録クラブではないが、更新として申請されている'
            else:
                print('申請種別が正しい')
        else:
            print(f"警告: クラブ名 '{club_name}' は申請内容を含むデータに見つかりませんでした。")
    else:
        print("警告: check_application_type 関数に空の application_row が渡されました。")
    return error_dict

In [ ]:
# 基準適合状況の入力を確認する関数
def check_standard_compliance(application_row):
    """
    基準適合状況について全て「適合している」となっているかをチェックします。
    （R8は(1)③については経過措置のためチェックを排除）
    """
    error_dict = {}
    if not application_row.empty:
        standard_column = [
            '基準適合状況(1)①：多種目（複数種目）のスポーツ活動を実施している',
            '基準適合状況 (1)  ②：多世代（複数世代）を対象としている',
            #'基準適合状況 (1)  ③：適切なスポーツ指導者を配置している', # R8ではチェックしない
            '基準適合状況 (1)  ④：安全管理体制を整備している',
            '基準適合状況（2）⑤：地域住民が主体的に運営している',
            '基準適合状況（3）⑥：規約等が意思決定機関の議決により整備され、当該規約等に基づいて運営している',
            '基準適合状況（3）⑦：事業計画・予算、事業報告・決算が、意思決定機関で議決されている'
            ]
        # 各カラムをループして '適合していない' が含まれているかを確認
        error_at_standard_column = [col for col in standard_column if not application_row.empty and application_row[col].iloc[0] == '適合していない']
        if error_at_standard_column:
             error_dict['e_a_005'] = f'登録基準に適合していない:{error_at_standard_column}'
        else:
            print('基準に適合している')
    else:
        print("警告: check_standard_compliance 関数に空のapplication_rowが渡されました。")
    return error_dict

In [ ]:
# 会員数の入力をチェックする関数
def check_number_of_members(application_row):
    """
    会員数に関するカラムに数字が記入されているかをチェックします。
    """
    error_dict = {}
    if not application_row.empty:
        num_of_members_column = [
            "「A未就学児」×「男性」の会員数を「数字」で入力してください",
            "「A未就学児」×「女性」の会員数を「数字」で入力してください",
            "「A未就学児」×「性別不明」の会員数を「数字」で入力してください",
            "「B小学生」×「男性」の会員数を「数字」で入力してください",
            "「B小学生」×「女性」の会員数を「数字」で入力してください",
            "「B小学生」×「性別不明」の会員数を「数字」で入力してください",
            "「C中学生」×「男性」の会員数を「数字」で入力してください",
            "「C中学生」×「女性」の会員数を「数字」で入力してください",
            "「C中学生」×「性別不明」の会員数を「数字」で入力してください",
            "「D高校生（～18歳）」×「男性」の会員数を「数字」で入力してください",
            "「D高校生（～18歳）」×「女性」の会員数を「数字」で入力してください",
            "「D高校生（～18歳）」×「性別不明」の会員数を「数字」で入力してください",
            "「E～29歳」×「男性」の会員数を「数字」で入力してください",
            "「E～29歳」×「女性」の会員数を「数字」で入力してください",
            "「E～29歳」×「性別不明」の会員数を「数字」で入力してください",
            "「F～39歳」×「男性」の会員数を「数字」で入力してください",
            "「F～39歳」×「女性」の会員数を「数字」で入力してください",
            "「F～39歳」×「性別不明」の会員数を「数字」で入力してください",
            "「G～49歳」×「男性」の会員数を「数字」で入力してください",
            "「G～49歳」×「女性」の会員数を「数字」で入力してください",
            "「G～49歳」×「性別不明」の会員数を「数字」で入力してください",
            "「H～59歳」×「男性」の会員数を「数字」で入力してください",
            "「H～59歳」×「女性」の会員数を「数字」で入力してください",
            "「H～59歳」×「性別不明」の会員数を「数字」で入力してください",
            "「I～69歳」×「男性」の会員数を「数字」で入力してください",
            "「I～69歳」×「女性」の会員数を「数字」で入力してください",
            "「I～69歳」×「性別不明」の会員数を「数字」で入力してください",
            "「J70歳～」×「男性」の会員数を「数字」で入力してください",
            "「J70歳～」×「女性」の会員数を「数字」で入力してください",
            "「J70歳～」×「性別不明」の会員数を「数字」で入力してください",
            ]
        num_of_annual_fee_members_column = [
            "「A未就学児」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「A未就学児」×「女性」の 年会費等を支払っている会員数を「数字」で入力してください",
            "「A未就学児」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「B小学生」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「B小学生」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「B小学生」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「C中学生」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「C中学生」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「C中学生」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「D高校生（～18歳）」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「D高校生（～18歳）」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「D高校生（～18歳）」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「E～29歳」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「E～29歳」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「E～29歳」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「F～39歳」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「F～39歳」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「F～39歳」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「G～49歳」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「G～49歳」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「G～49歳」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「H～59歳」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「H～59歳」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「H～59歳」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「I～69歳」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「I～69歳」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「I～69歳」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            "「J70歳～」×「男性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「J70歳～」×「女性」の年会費等を支払っている会員数を「数字」で入力してください",
            "「J70歳～」×「性別不明」の年会費等を支払っている会員数を「数字」で入力してください",
            ]

        error_at_num_of_members_column = []
        for col in num_of_members_column:
            # form_rowが空でないことと、値がNaNでないことを確認してから型をチェック
            if not application_row.empty and not pd.isna(application_row[col].iloc[0]) and not isinstance(application_row[col].iloc[0], (int, float)): # floatも許容する
                error_at_num_of_members_column.append(col)
        if error_at_num_of_members_column:
            error_dict['e_a_006-a'] = f'会員数の入力欄に数字ではないデータが入力されている:{error_at_num_of_members_column}'
        else:
            print('会員数の入力に問題が無い')

        error_at_num_of_annual_fee_members_column = []
        for col in num_of_annual_fee_members_column:
             # form_rowが空でないことと、値がNaNでないことを確認してから型をチェック
            if not application_row.empty and not pd.isna(application_row[col].iloc[0]) and not isinstance(application_row[col].iloc[0], (int, float)): # floatも許容する
                error_at_num_of_annual_fee_members_column.append(col)
        if error_at_num_of_annual_fee_members_column:
            error_dict['e_a_006-b'] = f'年会費等を支払っている会員数の入力欄に数字ではないデータが入力されている:{error_at_num_of_annual_fee_members_column}'
        else:
            print('年会費を払っている会員数の入力に問題が無い')
    else:
        print("警告: check_number_of_members 関数に空のapplication_rowが渡されました。")
    return error_dict


In [ ]:
# 定期的な活動が2種目以上かをチェックする関数
def check_number_of_disciplines(application_row):
    """
    定期的な活動が2種目以上かをチェックします。
    """
    error_dict = {}
    if not application_row.empty:
        # 種目のカラムをdisciplines_columnとして指定
        disciplines_column = [
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [アーチェリー]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [アイスホッケー]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [アメリカンフットボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [インディアカ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ウエイトリフティング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ウォーキング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [エアロビック]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [オリエンテーリング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [親子リトミック]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [カーリング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [カヌー]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [空手道]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [弓道]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [近代五種]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [キンボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [グラウンド・ゴルフ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [クレー射撃]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ゲートボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [健康体操]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [剣道]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ゴルフ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [サッカー]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [山岳]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [自転車競技]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [銃剣道]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [柔道]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [少林寺拳法]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [新体操]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [水泳(競泳・飛込・水球等)]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [スキー・スノーボード]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [スクーバ・ダイビング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [スケート]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [スポーツクライミング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [スポーツチャンバラ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [スポーツ吹矢]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [相撲]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [セーリング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ソフトテニス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ソフトバレーボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ソフトボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [太極拳]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [体操(一般体操)]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [体操競技]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [卓球]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ターゲット・バードゴルフ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ダンス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ダンススポーツ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [チアダンス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [チアリーディング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [綱引]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [テニス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ドッジボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [トライアスロン]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [トランポリン]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [なぎなた]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [軟式野球]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [バイアスロン]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [バウンドテニス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [馬術]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [バスケットボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [バドミントン]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [バレーボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [パワーリフティング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ハンドボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [フェンシング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [フットサル]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [フライングディスク]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [フラダンス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [プロゴルフ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [プロスキー]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [プロテニス]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ペタンク・ブール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ボウリング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ボート]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ボクシング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ホッケー]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ボブスレー・リュージュ・スケルトン]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [野球]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ヨガ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ライフル射撃]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ラグビーフットボール]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ランニング(ジョギング)]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [陸上競技]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [レスリング]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [ローラースポーツ]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [３Ｂ体操]",
            "定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [このリストに含まれていない種目]",
            '（上のリストにおいて、「このリストに含まれていない種目」を「実施している」と回答した方に伺います）「リストに含まれていない種目」のうち、定期的に行っているスポーツ活動種目の数'
        ]
        extra_disciplines_column = '（上のリストにおいて、「このリストに含まれていない種目」を「実施している」と回答した方に伺います）「リストに含まれていない種目」のうち、定期的に行っているスポーツ活動種目の数'
        # disciplines_columnのデータが「定期的に行っている」である数を数える
        count_of_disciplines = 0
        # 各種目カラムをループし、「定期的に行っている」が選択されているかを確認
        for col in disciplines_column:
            if col != extra_disciplines_column and not application_row.empty and application_row[col].iloc[0] == '定期的に行っている':
                count_of_disciplines += 1

        # リストに含まれていない種目について処理
        if not application_row.empty and application_row['定期的に行っているスポーツ活動種目を選択してください（※定期的とは年間12回以上行っていることを指します） [このリストに含まれていない種目]'].iloc[0] == '定期的に行っている':
            # リスト外の活動種目数の項目に数字が入力されているかをチェック
            extra_num = application_row[extra_disciplines_column].iloc[0]
            if pd.isna(extra_num) or not isinstance(extra_num, (int, float)): # floatも許容する
                error_dict['e_a_007-b'] = 'リスト外の活動種目数の項目に数字が入力されていない'
            else:
                count_of_disciplines += int(extra_num)

        # 2種目以上かを確認する
        if count_of_disciplines < 2:
            error_dict['e_a_007-a'] = '定期的に活動を2種目以上行っていない'
        else:
            print('活動種目の入力に問題は無い')
    else:
        print("警告: check_number_of_disciplines 関数に空のapplication_rowが渡されました。")
    return error_dict

In [ ]:
# 指導者の配置をチェックする関数（R8は、ほぼ機能なし）
def check_coaches(application_row):
  """
  指導者の配置状況をチェックします。
  """
  error_dict = {}
  # この関数は現段階では不要とのことなので、空の辞書を返します。
  # 必要になった場合は、ここに処理を追加してください。
  print('指導者の処理が終了しました（現段階では審査に不要のため）')
  return error_dict

In [ ]:
# マネジメント指導者資格の入力をチェックする関数
def check_managers(application_row):
  """
  クラマネ・アシマネの配置状況と人数の入力をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      # カラムを指定
      manager_column = [
          "クラブマネジャーのうち公認クラブマネジャー資格保有者の数を入力してください",
          "クラブマネジャーのうち公認アシスタントマネジャー資格保有者の数を入力してください",
          "事務局員のうち公認クラブマネジャー資格保有者の数を入力してください",
          "事務局員のうち公認アシスタントマネジャー資格保有者の数を入力してください"
      ]

      # まず配置有無を確認（配置していたら、次の処理へ）
      if form_row["クラブマネジャーの配置状況について入力してください（※クラブマネジャーとは経営能力を有する専門的な人材のことを指します）"].iloc[0] == "配置している":
          # manager_column に数字が入力されているかをチェック
          error_at_manager_num = []
          manager_num = 0
          for col in manager_column:
              value = application_row[col].iloc[0]
              if pd.isna(value) or not isinstance(value, (int, float)): # floatも許容する
                  error_at_manager_num.append(col)
              else:
                  manager_num += int(value)

          if error_at_manager_num:
              error_dict['e_a_009-a'] = f'マネジメント指導者数の入力欄に数字ではないデータが入力されている:{error_at_manager_num}'
          else:
              print('マネジメント指導者数の入力に問題が無い')

          # 現段階ではこれ以上の処理はしない
      else:
          print('マネジメント指導者は配置されていない')
  else:
      print("警告: check_managers 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# 規約等の改廃状況を確認する関数
def check_rule_revision(application_row):
  """
  規約等の改廃に関する提出書類をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      # 改廃状況を確認
      if application_row['前年度の申請以後における、規約等の改定状況について入力してください'].iloc[0] == '前年度の申請以降、改定をした':
          # 議事録が提出されているかを確認
          if not application_row['規約等の改廃を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_010-a'] = '規約等の改廃を決議した議事録が提出されていない'
          else:
              print('規約改廃時の議事録が提出されている')
          # 改正前の規約が提出されているかを確認
          if not application_row['改廃前の規約等をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_010-b'] = '改正前の規約等が提出されていない'
          else:
              print('規約改廃前の規約が提出されている')
      else:
          print('規約等の改廃は行われていない')
  else:
      print("警告: check_rule_revision 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# （現行の）規約等の提出確認をする関数
def check_rule_submission(application_row):
  """
  規約等の提出状況をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      # 提出の確認(新規登録なのに、提出していない場合はエラー)
      if application_row['規約等の提出有無について入力してください'].iloc[0] == '提出しない' and application_row['申請種別を選択してください'].iloc[0] == '新規（R7年度には登録していない）':
          error_dict['e_a_011-a'] = '新規登録なのに、規約等の提出がされていない'
      # 更新で、改正済みなのに、提出していない場合はエラー
      elif application_row['規約等の提出有無について入力してください'].iloc[0] == '提出しない' and application_row['申請種別を選択してください'].iloc[0] == '更新（R7年度に登録済み）' and application_row['前年度の申請以後における、規約等の改定状況について入力してください'].iloc[0] == '前年度の申請以降、改定をした':
          error_dict['e_a_011-b'] = '更新で、改正済みなのに、提出していない'
      # 提出する場合は、提出されているかを確認
      elif application_row['規約等の提出有無について入力してください'].iloc[0] == '提出する':
          # 提出状況の確認
          if not application_row['規約等をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_011-c'] = '規約等の提出がされていない'
          else:
              print('規約の提出に問題が無い')
      else:
          print('規約は提出されていないが、更新のため問題は無い（前年度に提出された議事録に問題が無いかを確認する必要がある）')
          # ここに、手動での確認表に追記するような処理を挿入する
  else:
      print("警告: check_rule_submission 関数に空のapplication_rowが渡されました。")
  return error_dict


In [ ]:
# 役員名簿（議決権保有者名簿）の提出をチェックする関数
def check_officer_list_submission(application_row):
  """
  役員名簿（議決権保有者名簿）の提出状況をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      # 提出の確認(新規登録なのに、提出していない場合はエラー)
      if application_row['役員名簿（議決権保有者名簿）  の提出有無について入力してください'].iloc[0] == '提出しない' and application_row['申請種別を選択してください'].iloc[0] == '新規（R7年度には登録していない）':
          error_dict['e_a_012-a'] = '新規登録なのに、役員名簿（議決権保有者名簿）の提出がされていない'
      # 更新で、改正済みなのに、提出していない場合はエラー
      elif application_row['役員名簿（議決権保有者名簿）  の提出有無について入力してください'].iloc[0] == '提出しない' and application_row['申請種別を選択してください'].iloc[0] == '更新（R7年度に登録済み）':
          error_dict['e_a_012-b'] = '更新なのに、提出していない'
      # 提出する場合は、提出されているかを確認
      elif application_row['役員名簿（議決権保有者名簿）  の提出有無について入力してください'].iloc[0] == '提出する':
          # 提出状況の確認
          if not application_row['役員名簿（議決権保有者名簿）をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_012-c'] = '役員名簿（議決権保有者名簿）の提出がされていない'
          else:
              print('役員名簿の提出に問題が無い')
  else:
      print("警告: check_officer_list_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# 事業計画とそれを決議した議事録の提出状況を確認する関数
def check_business_plan_submission(application_row):
  """
  事業計画と決議した議事録の提出状況をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      # 事業計画が提出されているかを確認
      if not application_row['決議された事業計画をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
          error_dict['e_a_013-a'] = '事業計画が提出されていない'
      else:
          print('事業計画が提出されている')
      # 事業計画を決議した際の議事録が提出されているかを確認
      if not application_row['事業計画を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
          error_dict['e_a_013-b'] = '事業計画を決議した議事録が提出されていない'
      else:
          print('事業計画を決議した議事録が提出されている')
  else:
      print("警告: check_business_plan_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# 予算とそれを決議した議事録の提出状況を確認する関数
def check_budget_submission(application_row):
  """
  予算と決議した議事録の提出状況をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      # 予算が提出されているかを確認
      if not application_row['決議された予算をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
          error_dict['e_a_014-a'] = '予算が提出されていない'
      else:
          print('予算が提出されている')
      # 予算を決議した際の議事録が提出されているかを確認
      if not application_row['予算を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
          error_dict['e_a_014-b'] = '予算を決議した議事録が提出されていない'
      else:
          print('予算を決議した議事録が提出されている')
  else:
      print("警告: check_budget_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# 事業報告とそれを決議した議事録の提出状況を確認する関数
def check_business_report_submission(application_row):
  """
  事業報告と決議した議事録の提出状況をチェックします。
  今年度新設クラブの場合は免除されます。
  """
  error_dict = {}
  if not application_row.empty:
      # 今年度新設クラブは未提出が許されるので、未提出かつ新設かをチェック
      today = datetime.date.today()
      one_year_ago = today - datetime.timedelta(days=365) # 厳密には年度で判定すべきですが、ここでは1年前としています

      is_new_club = False
      if not pd.isna(application_row['クラブの設立日を入力してください'].iloc[0]):
          try:
              establishment_date = pd.to_datetime(application_row['クラブの設立日を入力してください'].iloc[0]).date()
              if establishment_date >= one_year_ago:
                  is_new_club = True
          except ValueError:
              print(f"警告: クラブ設立日のフォーマットが不正です: {application_row['クラブの設立日を入力してください'].iloc[0]}")

      if application_row['事業報告の提出有無について入力してください'].iloc[0] == '今年度新設されたクラブのため、提出しない' and not is_new_club:
          error_dict['e_a_015-a'] = '昨年度以前の設立クラブだが、事業報告が提出されていない'
      # 提出がなされているかをチェック
      elif application_row['事業報告の提出有無について入力してください'].iloc[0] == '提出する':
          if not application_row['決議された事業報告をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_015-b'] = '事業報告が提出されていない'
          else:
              print('事業報告が提出されている')
          if not application_row['事業報告を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_015-c'] = '事業報告を決議した議事録が提出されていない'
          else:
              print('事業報告を決議した議事録が提出されている')
      else:
          print('事業報告は提出されていないが、今年度新設クラブのため問題は無い（処理にエラーがある可能性や、クラブの事業年度上は2年目に突入している可能性を確認する必要がある）')
          # ここに、手動での確認表に追記するような処理を挿入する
  else:
      print("警告: check_business_report_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# 決算とそれを決議した議事録の提出状況を確認する関数
def check_financial_statement_submission(application_row):
  """
  決算と決議した議事録の提出状況をチェックします。
  今年度新設クラブの場合は免除されます。
  """
  error_dict = {}
  if not application_row.empty:
      # 今年度新設クラブは未提出が許されるので、未提出かつ新設かをチェック
      today = datetime.date.today()
      one_year_ago = today - datetime.timedelta(days=365) # 厳密には年度で判定すべきですが、ここでは1年前としています

      is_new_club = False
      if not pd.isna(application_row['クラブの設立日を入力してください'].iloc[0]):
          try:
              establishment_date = pd.to_datetime(application_row['クラブの設立日を入力してください'].iloc[0]).date()
              if establishment_date >= one_year_ago:
                  is_new_club = True
          except ValueError:
              print(f"警告: クラブ設立日のフォーマットが不正です: {application_row['クラブの設立日を入力してください'].iloc[0]}")

      if application_row['決算の提出有無について入力してください'].iloc[0] == '今年度新設されたクラブのため、提出しない' and not is_new_club:
          error_dict['e_a_016-a'] = '昨年度以前の設立クラブだが、決算が提出されていない'
      elif application_row['決算の提出有無について入力してください'].iloc[0] == '提出する':
          if not application_row['決算をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_016-b'] = '決算が提出されていない'
          else:
              print('決算が提出されている')
          if not application_row['決算を決議した際の議事録をアップロードしてください※同一の書類を提出済みでも、提出をお願いします。'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
              error_dict['e_a_016-c'] = '決算を決議した議事録が提出されていない'
          else:
              print('決算を決議した議事録が提出されている')
      else:
          print('決算は提出されていないが、今年度新設クラブのため問題は無い（処理にエラーがある可能性や、クラブの事業年度上は2年目に突入している可能性を確認する必要がある）')
          # ここに、手動での確認表に追記するような処理を挿入する
  else:
      print("警告: check_financial_statement_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# チェックシートの提出状況を確認する関数
def check_checklist_submission(application_row):
  """
  チェックシートの提出状況をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      if not application_row['評価指標を用いたクラブの自己点検・評価シートをアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
          error_dict['e_a_017-a'] = 'チェックシートが提出されていない'
      else:
          print('チェックシートが提出されている')
          # 内容に問題が無いかをチェックする機能を今後追加
  else:
      print("警告: check_checklist_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# 自己説明・公表確認書の提出状況を確認する関数
def check_self_explanation_submission(application_row):
  """
  自己説明・公表確認書の提出状況をチェックします。
  """
  error_dict = {}
  if not application_row.empty:
      if not application_row['自己説明・公表確認書をアップロードしてください'].iloc[0]: # Googleフォームのファイルのアップロード項目はファイルがアップロードされるとFalse以外になることを想定
          error_dict['e_a_018-a'] = '自己説明・公表確認書が提出されていない'
      else:
          print('自己説明・公表確認書が提出されている')
          # 内容に問題が無いかをチェックする機能を今後追加
  else:
      print("警告: check_self_explanation_submission 関数に空のapplication_rowが渡されました。")
  return error_dict

In [ ]:
# クラブごとの自動チェックを行う関数
def perform_automatic_checks(club_name, application_data_df, checklist_folder_path, checklist_create_df, timestamp):
    """
    指定されたクラブに対して自動チェックを行います。

    Args:
        club_name (str): チェックを行うクラブ名。
        application_data_df (pd.DataFrame): 申請内容を含むDataFrame（マージ済みの club_list_df）。
        checklist_folder_path (str): チェックリストが保存されているフォルダパス。
        checklist_create_df (pd.DataFrame): チェックリスト作成状況を含むDataFrame。
        timestamp (str): 現在のタイムスタンプ（yyyymmddHHMMSS形式）。

    Returns:
        None: チェック結果はチェックリストファイルに書き込まれます。
    """
    print(f"クラブ名: {club_name}")

    # 申請日時を取得（checklist_create_df から）
    # checklist_create_df にクラブ名が存在することを確認
    if club_name not in checklist_create_df['クラブ名'].values:
        print(f"警告: クラブ名 '{club_name}' はチェックリスト作成状況に見つかりませんでした。このクラブの処理をスキップします。")
        return

    application_date_str_from_checklist = checklist_create_df.loc[checklist_create_df['クラブ名'] == club_name, '申請日時'].iloc[0]
    checklist_creation_date_str = checklist_create_df.loc[checklist_create_df['クラブ名'] == club_name, 'チェックリスト作成日時'].iloc[0]

    print(f"申請日時 (from checklist): {application_date_str_from_checklist}")
    print(f"チェックリスト作成日時: {checklist_creation_date_str}")

    # 対象クラブのチェックリストを読み込み
    # row['申請日時']をyyyymmddhhmmssに変換
    try:
        # checklist_create_dfに保存されている申請日時が文字列の場合を考慮
        if isinstance(application_date_str_from_checklist, str):
             application_date_from_checklist = datetime.strptime(application_date_str_from_checklist, '%Y-%m-%d %H:%M:%S')
        else: # datetime型の場合
             application_date_from_checklist = application_date_str_from_checklist
        application_date_str_yyyymmddHHMMSS = application_date_from_checklist.strftime('%Y%m%d%H%M%S')
    except ValueError:
        print(f"エラー: クラブ '{club_name}' の申請日時のフォーマットが不正です: {application_date_str_from_checklist}")
        return

    checklist_file_name = f"{club_name}_申請{application_date_str_yyyymmddHHMMSS}_作成{checklist_creation_date_str}.csv"
    checklist_file_path = os.path.join(checklist_folder_path, checklist_file_name)
    print(f"チェックリストの読み込みを開始しました: {club_name}")

    # チェックリストファイルが存在するかどうかを確認
    if not os.path.exists(checklist_file_path):
        print(f"警告: チェックリストファイル '{checklist_file_path}' が見つかりませんでした。このクラブの処理をスキップします。")
        return

    try:
        checklist_df = pd.read_csv(checklist_file_path)
    except Exception as e:
        print(f"エラー: チェックリストファイル '{checklist_file_path}' の読み込み中にエラーが発生しました: {e}")
        return
    print('チェックリストファイルが見つかりました')

    # application_data_dfから現在のクラブの行を取得
    print(application_data_df)
    print(application_data_df.head)
    print(application_data_df['クラブ名'])
    application_row = application_data_df.loc[application_data_df['クラブ名'] == club_name]

    if application_row.empty:
        print(f"警告: クラブ名 '{club_name}' は申請内容を含むデータに見つかりませんでした。このクラブの処理をスキップします。")
        return
    else:
        print('クラブ名が申請内容に存在')

    # 入力エラーが無いかを記載していくための辞書を作成（まずは空）
    error_dict = {}

    # チェック項目を関数に分割し、呼び出す
    # 各チェック関数に application_row を渡すように変更
    error_dict.update(check_must_columns(application_row))
    error_dict.update(check_submitting_now(application_row))
    error_dict.update(check_phone_number(application_row))
    error_dict.update(check_fax_number(application_row))
    # check_application_type 関数には application_data_df を渡す
    error_dict.update(check_application_type(application_row, application_data_df, club_name))
    error_dict.update(check_standard_compliance(application_row))
    error_dict.update(check_number_of_members(application_row))
    error_dict.update(check_number_of_disciplines(application_row))
    # error_dict.update(check_coaches(application_row)) # 現段階では不要
    error_dict.update(check_managers(application_row))
    error_dict.update(check_rule_revision(application_row))
    error_dict.update(check_rule_submission(application_row))
    error_dict.update(check_officer_list_submission(application_row))
    error_dict.update(check_business_plan_submission(application_row))
    error_dict.update(check_budget_submission(application_row))
    error_dict.update(check_business_report_submission(application_row))
    error_dict.update(check_financial_statement_submission(application_row))
    error_dict.update(check_checklist_submission(application_row))
    error_dict.update(check_self_explanation_submission(application_row))


    # 一旦、チェックリストをprint
    print(f"チェック結果: {error_dict}")

    # 自動でチェックした内容をチェックリストに書き出し(チェックリストの上書き）
    # error_dictが空の場合は空文字列を書き込む
    checklist_df['自動チェック'] = str(error_dict) if error_dict else ''
    checklist_df['自動チェック更新時間'] = datetime.now(timezone(timedelta(hours=9))).strftime('%Y-%m-%d %H:%M:%S')

    # 追記したカラムをチェック
    print(checklist_df[['自動チェック','自動チェック更新時間']].iloc[0])

    # チェックリストファイルを保存
    try:
        checklist_df.to_csv(checklist_file_path, index=False)
        print(f"チェックリストファイル '{checklist_file_path}' を更新しました。")
    except Exception as e:
        print(f"エラー: チェックリストファイル '{checklist_file_path}' の書き込み中にエラーが発生しました: {e}")

In [ ]:
# チェック処理の実行部分
# 必要なデータを読み込み
checklist_create_df = pd.read_csv('R7_登録申請処理/申請入力内容/クラブごとのチェックリスト作成状況.csv')

# クラブリストを読み込み
# クラブリスト_{timestamp}.csvを読み込む
# folderを参照して、最新のクラブリストを探す
foldr_path = 'R7_登録申請処理/クラブリスト'
latest_file = max([os.path.join(foldr_path, f) for f in os.listdir(foldr_path)], key=os.path.getctime)

# 最新のクラブリストを読み込む
club_list_df = pd.read_csv(latest_file)

try:
    club_list_df = pd.read_csv(latest_file)
    # タイムスタンプをdatetime型に変換
    club_list_df['タイムスタンプ'] = pd.to_datetime(club_list_df['タイムスタンプ'], errors='coerce')
    club_list_df['タイムスタンプyyyymmddHHMMSS'] = club_list_df['タイムスタンプ'].dt.strftime('%Y%m%d%H%M%S')
    print("申請内容の読み込みが完了しました")
except Exception as e:
    print(f"申請内容の読み込み中にエラーが発生しました: {e}")

# チェックリストのフォルダを指定
checklist_folder_path = 'R7_登録申請処理/申請入力内容'

# 日本の現在時刻を取得
jst_now = datetime.now(timezone(timedelta(hours=9)))

# 時刻をyyyymmddHHMMSS形式に変換
timestamp = jst_now.strftime("%Y%m%d%H%M%S")

# クラブリストの各行に対して自動チェックを実行
# checklist_create_df をイテレートして、各クラブに対してチェックを実行
for index, row in checklist_create_df.iterrows():
    club_name = row['クラブ名']
    perform_automatic_checks(club_name, club_list_df, checklist_folder_path, checklist_create_df, timestamp)

In [ ]:
# 人間がチェックする用のリストを作成
## 申請内容
## 書類ごとの内容

# 実装するべき機能
# 更新かつ、規約等改正無しかつ、規約等の提出が無い場合は、チェック項目を追加
# 事業報告・決算を新設としたクラブは、設立年と事業年度をチェック

## 処理間の整合性
print('とりあえず処理が終了しました～☺')